# <CENTER> DATA DE CRM

## *PASOS INICIALES*

### *LIBRERÍAS*

In [2]:
import pandas as pd
import numpy as np
import unidecode
import os
import openpyxl
import datetime
from datetime import date, datetime, timedelta
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
import re

pd.set_option('display.max_columns', None)

### *RUTAS*

In [51]:
ruta_personas = '../data/crm/00_personas_leads_contacts_opps.parquet'
ruta_leads = '../data/crm/01_leads.parquet'
ruta_contact = '../data/crm/02_contact.parquet'
ruta_tipificaciones = '../data/crm/10_tipificaciones.parquet'

ruta_output_pq = '../output/ds_crm.parquet'

### *FUNCIONES*

### *DATOS A ESPECIFICAR*

In [7]:
dict_fechas_inicio_periodo = {
    202401 : datetime( 2024 , 3 , 25 ) ,
    202402 : datetime( 2024 , 8 , 12 ) ,
    202501 : datetime( 2025 , 3 , 24 ) ,
    202502 : datetime( 2025 , 8 , 18 ) ,
}

dict_fechas_fin_periodo = {
    202401 : datetime( 2024 , 7 , 26 ) ,
    202402 : datetime( 2024 , 12 , 12 ) ,
    202501 : datetime( 2025 , 7 , 25 ) ,
    202502 : datetime( 2025 , 12 , 19 ) ,
}

lista_periodos = sorted( list( dict_fechas_inicio_periodo.keys() ) )

# CARGAR DATA

## PERSONAS

In [11]:
df_pers = pd.read_parquet( ruta_personas )

df_pers

,persona_id,documento,documento_validado,telefono,lead_id,contact_id,opp_id,nombre_completo,lista_nombres_validos,campo_homologacion,flag_persona_valida
0,DOC-00001008,00001008,00001008,None,0BC4DB79-20D8-ED11-A7C6-002248E05014,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
1,DOC-00001008,00001008,00001008,None,1C479AD6-5E4B-EE11-BE6F-002248E0D695,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
2,DOC-00001008,00001008,00001008,None,38691934-C72C-ED11-9DB1-000D3A89214D,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
3,DOC-00001008,00001008,00001008,None,3DA0B9C1-0A8B-F011-B4CC-002248DEB3C7,BFCF517A-A91C-F011-998A-6045BD3BDCD7,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
4,DOC-00001008,00001008,00001008,None,409CC8A9-BAFE-EE11-9F8A-000D3AC16AC8,None,None,SARA MIRANDA ATIQUIPA,[SARA MIRANDA ATIQUIPA],documento,1
...,...,...,...,...,...,...,...,...,...,...,...
14872588,CID-A92279C4-AAEF-F011-8407-7CED8DB0C240,None,None,None,None,A92279C4-AAEF-F011-8407-7CED8DB0C240,None,PRUEBA DEV AMBIENTE DEV AMBIENTE,[PRUEBA DEV AMBIENTE DEV AMBIENTE],nd,0
14872589,CID-DB3EA2A7-ADF7-F011-8406-002248DE7A19,None,None,None,None,DB3EA2A7-ADF7-F011-8406-002248DE7A19,None,TEST TEST TEST,[TEST TEST TEST],nd,0
14872590,CID-9B65E971-E9FD-F011-8407-002248DEDFD1,None,None,None,None,9B65E971-E9FD-F011-8407-002248DEDFD1,None,PRUEBA PRUEBA PRUEBA,[PRUEBA PRUEBA PRUEBA],nd,0
14872591,CID-E83F695F-29FE-F011-8407-002248DE712B,None,None,None,None,E83F695F-29FE-F011-8407-002248DE712B,None,PRUEBA PRUEBA PRIEBA,[PRUEBA PRUEBA PRIEBA],nd,0


## LEADS

In [10]:
df_leads = pd.read_parquet( ruta_leads )

df_leads

,lead_id,opp_id,contact_id,fecha_lead,unidad_negocio,campana,modalidad_estudios,fuente,fuente_detalle,medio_publicitario,medio_publicitario_detalle,campus,carrera,tipo_ingreso,subtipo_ingreso,documento,nombre_completo,nombres,apellido_paterno,apellido_materno,telefono,otro_telefono_1,otro_telefono_2,otro_telefono_3,correo_1,correo_2,correo_3,file_name
0,D38785AF-E639-EC11-8C62-000D3A88605F,None,None,2021-10-30,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,INGENIERIA DE SISTEMAS COMPUTACIONALES,None,None,76919962,LISETH ARACELI HUAMAN ESCOBAR,LISETH ARACELI,HUAMAN,ESCOBAR,998586349,None,None,None,lisetnahuam@gmail.com,None,NaN,leads_00000000_20211231.parquet
1,3C224550-EA39-EC11-8C62-000D3A88605F,None,DC02807E-58A2-ED11-AAD1-000D3A892201,2021-10-30,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,DERECHO Y CIENCIAS POLITICAS,None,None,73323774,JOSE AUGUSTO ALIAGA BAZAN,JOSE AUGUSTO,ALIAGA,BAZAN,955755276,None,None,None,aliagajose342@gmail.com,None,NaN,leads_00000000_20211231.parquet
2,47E6A80A-ED39-EC11-8C62-000D3A88605F,66D78260-299E-EC11-B400-002248D30732,64D78260-299E-EC11-B400-002248D30732,2021-10-30,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,INGENIERIA CIVIL,None,None,70403975,HARBEY DARLYN CARRERA CALDERON,HARBEY,CARRERA,CALDERON,940178162,None,None,None,hcc_t_@hotmail.com,None,NaN,leads_00000000_20211231.parquet
3,C30F4091-413A-EC11-8C62-000D3A88605F,None,None,2021-10-31,PRE-GRADO,2022-01 PG,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,DERECHO Y CIENCIAS POLITICAS,None,None,47239704,WILMAN JULCAMORO CUEVA,WILMAN,JULCAMORO,CUEVA,914268741,None,None,None,wilmanjulcamorocueva@gmail.com,None,NaN,leads_00000000_20211231.parquet
4,D653993D-6C3A-EC11-8C62-000D3A88605F,None,C98D047E-CBA3-EC11-9841-002248DEA32F,2021-10-31,WORKING ADULT,2022-01 WA,None,WEB,HOMEPAGE INSTITUCION,POR DEFECTO,POR DEFECTO,CAJAMARCA,WA DERECHO Y CIENCIAS POLITICAS,None,None,45954022,ELMER ORRILLO DAVAN,ELMER,ORRILLO,DAVAN,994827568,976016680,None,None,eoledstg2311@gmail.com,None,NaN,leads_00000000_20211231.parquet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14550651,FF08743A-C0FE-F011-8406-002248DE7A19,None,8CEB75D2-BFFE-F011-8406-002248DE7A19,2026-01-31,WORKING ADULT,2026-01 WA,A DISTANCIA,WEB,HOMEPAGE INSTITUCION,None,None,WAONLINE,WV DERECHO,None,None,40022956,FELIPE GONZALES SUYO,FELIPE,GONZALES,SUYO,952521354,None,None,None,gonzafelipe74@gmail.com,None,NaN,leads_20260126_20260201.parquet
14550652,FF3E12BB-BFFE-F011-8407-7CED8DA81CCF,None,4FCA2F0F-33A8-EF11-A72D-00224837906C,2026-01-31,EPEC,2026 MA-18M 25-ABR,None,WEB,HOMEPAGE INSTITUCION,None,None,WAONLINE,MAESTRIA EN GESTION DE LA CALIDAD Y GERENCIA D...,None,None,45842621,EDER BALLESTEROS TORRES,EDER,BALLESTEROS,TORRES,947298717,None,None,None,ederballesteros@gmail.com,None,NaN,leads_20260126_20260201.parquet
14550653,FF51D012-47FE-F011-8407-002248DED09F,None,45C5E018-47FE-F011-8407-002248DED09F,2026-01-30,WORKING ADULT,2026-01 WA,SEMI PRESENCIAL,WEB,HOMEPAGE INSTITUCION,None,None,CHORRILLOS,WS PSICOLOGIA,None,None,46028442,CATHERINE PUMACAYO ZUMAETA,CATHERINE,PUMACAYO,ZUMAETA,941675933,None,None,None,crazy_8914@hotmail.com,None,NaN,leads_20260126_20260201.parquet
14550654,FF789541-F8FD-F011-8407-002248DED024,None,5485D6A6-2E1C-F011-998A-6045BD3C5144,2026-01-30,EPEC,2026 MA-18M 25-ABR,None,WEB,HOMEPAGE INSTITUCION,None,None,WAONLINE,MAESTRIA EN DERECHO CORPORATIVO Y GESTION EMPR...,None,None,71028855,NORMA MARGARITA MENDIBURU ALVAN,NORMA MARGARITA,MENDIBURU,ALVAN,904608152,None,None,None,mmendiburu13@gmail.com,None,NaN,leads_20260126_20260201.parquet


## CONTACT

In [5]:
df_contact = pd.read_parquet( ruta_contact )

df_contact

,contact_id,fecha_contact,documento,nombre_completo,nombres,apellido_paterno,apellido_materno,telefono,otro_telefono_1,otro_telefono_2,otro_telefono_3,correo_1,correo_2,correo_3,correo_upn_temp,cod_alumno,file_name
0,60C63BDF-FA3B-EC11-8C62-000D3A88605F,2021-11-02,75555921,CARLOS FERNANDO BECERRA ZAMORA,CARLOS,BECERRA,ZAMORA,917192599,None,None,None,N00320186@upn.pe,carlosbz2308@gmail.com,None,N00320186@upn.pe,N00320186,contact_00000000_20250316.parquet
1,C6FDBCF8-143C-EC11-8C62-000D3A88605F,2021-11-02,41155941,PERCY ROJAS CHAVEZ,PERCY,ROJAS,CHAVEZ,917631564,None,None,None,jhoxisrocha@gmail.com,None,None,None,None,contact_00000000_20250316.parquet
2,3EE94091-FD3B-EC11-8C62-000D3A88605F,2021-11-02,43674916,OSCAR REYES VASQUEZ,OSCAR,REYES,VASQUEZ,956218779,None,None,None,N00031822@upn.pe,OSCAREYESV@HOTMAIL.COM,None,N00031822@upn.pe,N00031822,contact_00000000_20250316.parquet
3,7E6AC9DD-043C-EC11-8C62-000D3A88605F,2021-11-02,74318245,JHOSEP ALEXIS ROJAS CHAVEZ,JHOSEP,ROJAS,CHAVEZ,917631564,None,None,None,N00320215@upn.pe,jhoxisrocha@gmail.com,None,N00320215@upn.pe,N00320215,contact_00000000_20250316.parquet
4,24251033-143C-EC11-8C62-000D3A88605F,2021-11-02,73635913,YULY ROJAS ESTELA,YULY,ROJAS,ESTELA,922513036,None,None,None,Yuca_11@hotmail.com,None,None,N00323167@upn.pe,N00323167,contact_00000000_20250316.parquet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2180250,2C7F1AC1-9EFD-F011-8407-000D3AC1A0F2,2026-01-30,70293126,CORDOVA HERNAN,CORDOVA,HERNAN,None,None,None,None,None,pana_hernan@hotmail.com,None,None,None,None,contact_20260126_20260201.parquet
2180251,99041DF9-9FFD-F011-8407-000D3AC1A0F2,2026-01-30,48202456,DIANA RAMOS TORRES,DIANA,RAMOS,TORRES,980340145,None,None,None,dilirato2102@gmail.com,None,None,None,None,contact_20260126_20260201.parquet
2180252,CF685D1C-ADFA-F011-8406-000D3AC1A0F2,2026-01-26,05008995,MARIA ANTONIETA MORA SALAS,MARIA ANTONIETA,MORA,SALAS,910104344,None,None,None,ANTONIETA.MORA.21@GMAIL.COM,None,None,None,None,contact_20260126_20260201.parquet
2180253,42C0C275-B6FA-F011-8406-000D3AC1A0F2,2026-01-26,48867488,EDITH HUARCAYA YAURI,EDITH,HUARCAYA YAURI,None,935952068,None,None,None,None,None,None,None,None,contact_20260126_20260201.parquet


## TIPIFICACIONES

In [6]:
df_tipif = pd.read_parquet( ruta_tipificaciones )

df_tipif

,persona_id,flag_persona_valida,fecha_tipificacion,cant_tipif,cant_tipif_comerc,cant_tipif_no_comerc,cant_tipif_contact,cant_tipif_comerc_contact,cant_tipif_contact_efect,cant_tipif_comerc_contact_efect,suma_ptje_interes_tipif,suma_ptje_interes_tipif_comerc,suma_ptje_interes_tipif_comerc_contact,suma_ptje_interes_tipif_comerc_contact_efect,max_ptje_interes_tipif,max_ptje_interes_tipif_comerc,max_ptje_interes_tipif_comerc_contact,max_ptje_interes_tipif_comerc_contact_efect
0,DOC-00001008,1,2023-06-09,1,1,0,1,1,1,1,3,3,3,3,3,3,3,3
1,DOC-00001008,1,2023-06-10,1,1,0,1,1,1,1,3,3,3,3,3,3,3,3
2,DOC-00001008,1,2023-06-17,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0
3,DOC-00001008,1,2023-07-04,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0
4,DOC-00001008,1,2023-07-17,2,2,0,2,2,2,2,6,6,6,6,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21983824,TLF-999989548,1,2025-03-12,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0
21983825,TLF-999989830,1,2023-09-18,2,2,0,0,0,0,0,2,2,0,0,1,1,0,0
21983826,TLF-999989830,1,2023-09-19,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0
21983827,TLF-999989830,1,2023-09-22,2,2,0,0,0,0,0,2,2,0,0,1,1,0,0


# TRATAMIENTO DE DATA

In [26]:
df_leads_periodo = pd.merge( df_leads[ [ 'lead_id' , 'opp_id' , 'contact_id' , 'fecha_lead' , 'documento' ] ] , df_contact[ [ 'contact_id' , 'documento' , 'cod_alumno' ] ] , how = 'left' , on = [ 'contact_id' ] )
df_leads_periodo['documento'] = df_leads_periodo['documento_x'].fillna( df_leads_periodo['documento_y'] )

df_leads_periodo = pd.merge( df_leads_periodo[ [ 'documento' , 'cod_alumno' , 'lead_id' , 'fecha_lead' ] ] , df_pers[ [ 'lead_id' , 'persona_id' ] ] , how = 'left' , on = [ 'lead_id' ] )

df_pers_alumno = df_leads_periodo[ [ 'persona_id' , 'documento' , 'cod_alumno' ] ].copy()
df_pers_alumno = df_pers_alumno.drop_duplicates().reset_index( drop = True )

lista_dfs = []
for periodo in lista_periodos :
    fecha_inicio_periodo = dict_fechas_inicio_periodo[periodo]
    df_leads_periodo_temp = df_leads_periodo[ df_leads_periodo['fecha_lead'] < fecha_inicio_periodo ].copy()
    df_leads_periodo_temp['periodo'] = periodo
    df_leads_periodo_temp = df_leads_periodo_temp.groupby( [ 'periodo' , 'persona_id' ] , as_index = False ).agg(
        cant_lead_ids = ( 'lead_id' , 'nunique' ) ,
    )
    lista_dfs.append( df_leads_periodo_temp )
df_leads_periodo = pd.concat( lista_dfs , axis = 0 ).reset_index( drop = True )

display( df_pers_alumno )
display( df_leads_periodo )

,persona_id,documento,cod_alumno
0,DOC-76919962,76919962,NaN
1,DOC-73323774,73323774,None
2,DOC-70403975,70403975,N00018892
3,DOC-47239704,47239704,NaN
4,DOC-45954022,45954022,N00029548
...,...,...,...
4137019,DOC-72273261,75557436,None
4137020,DOC-73941464,73941464,None
4137021,DOC-48073443,48073443,N00542516
4137022,DOC-06074328,06074328,None


,periodo,persona_id,cant_lead_ids
0,202401,DOC-00001008,4
1,202401,DOC-00001085,3
2,202401,DOC-00001100,1
3,202401,DOC-00001110,4
4,202401,DOC-00001177,1
...,...,...,...
10951547,202502,TLF-999989549,7
10951548,202502,TLF-999989677,7
10951549,202502,TLF-999989830,7
10951550,202502,TLF-999989887,7


In [38]:
df_tipif_periodo = df_tipif[ [ 'persona_id' , 'fecha_tipificacion' , 'cant_tipif' , 'cant_tipif_contact_efect' , 'cant_tipif_no_comerc' , 'cant_tipif_comerc' , 'cant_tipif_comerc_contact_efect' ] ].copy()
df_tipif_periodo = df_tipif_periodo[ df_tipif_periodo['persona_id'].isin( df_pers_alumno['persona_id'].unique().tolist() ) ].copy().reset_index( drop = True )

df_tipif_periodo['fecha_tipif_no_comerc'] = np.where( df_tipif_periodo['cant_tipif_no_comerc'] == 1 , df_tipif_periodo['fecha_tipificacion'] , np.datetime64('nat') )
df_tipif_periodo['fecha_tipif_comerc_contact_efect'] = np.where( df_tipif_periodo['cant_tipif_comerc_contact_efect'] == 1 , df_tipif_periodo['fecha_tipificacion'] , np.datetime64('nat') )

lista_dfs = []
for periodo in lista_periodos :
    fecha_inicio_periodo = dict_fechas_inicio_periodo[periodo]
    df_tipif_periodo_temp = df_tipif_periodo[ df_tipif_periodo['fecha_tipificacion'] < fecha_inicio_periodo ].copy()
    df_tipif_periodo_temp['periodo'] = periodo
    df_tipif_periodo_temp = df_tipif_periodo_temp.groupby( [ 'periodo' , 'persona_id' ] , as_index = False ).agg(
        cant_tipif = ( 'cant_tipif' , 'sum' ) ,
        cant_tipif_contact_efect = ( 'cant_tipif_contact_efect' , 'sum' ) ,
        cant_tipif_no_comerc = ( 'cant_tipif_no_comerc' , 'sum' ) ,
        cant_tipif_comerc = ( 'cant_tipif_comerc' , 'sum' ) ,
        cant_tipif_comerc_contact_efect = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
        fecha_prim_tipif = ( 'fecha_tipificacion' , 'min' ) ,
        fecha_ult_tipif = ( 'fecha_tipificacion' , 'max' ) ,
        fecha_prim_tipif_no_comerc = ( 'fecha_tipif_no_comerc' , 'min' ) ,
        fecha_ult_tipif_no_comerc = ( 'fecha_tipif_no_comerc' , 'max' ) ,
        fecha_prim_tipif_comerc_contact_efect = ( 'fecha_tipif_comerc_contact_efect' , 'min' ) ,
        fecha_ult_tipif_comerc_contact_efect = ( 'fecha_tipif_comerc_contact_efect' , 'max' ) ,
    )
    lista_dfs.append( df_tipif_periodo_temp )
df_tipif_periodo = pd.concat( lista_dfs , axis = 0 ).reset_index( drop = True )

df_tipif_periodo['ratio_tipif_contact_efect'] = df_tipif_periodo['cant_tipif_contact_efect'] /df_tipif_periodo['cant_tipif']
df_tipif_periodo['ratio_tipif_comerc'] = df_tipif_periodo['cant_tipif_comerc'] /df_tipif_periodo['cant_tipif']
df_tipif_periodo['ratio_tipif_comerc_contact_efect'] = df_tipif_periodo['cant_tipif_comerc_contact_efect'] /df_tipif_periodo['cant_tipif_comerc']

df_tipif_periodo

,periodo,persona_id,cant_tipif,cant_tipif_contact_efect,cant_tipif_no_comerc,cant_tipif_comerc,cant_tipif_comerc_contact_efect,fecha_prim_tipif,fecha_ult_tipif,fecha_prim_tipif_no_comerc,fecha_ult_tipif_no_comerc,fecha_prim_tipif_comerc_contact_efect,fecha_ult_tipif_comerc_contact_efect,ratio_tipif_contact_efect,ratio_tipif_comerc,ratio_tipif_comerc_contact_efect
0,202401,DOC-00001008,17,9,0,17,9,2023-06-09,2023-10-24,NaT,NaT,2023-06-09,2023-09-29,0.529412,1.0,0.529412
1,202401,DOC-00001085,5,0,0,5,0,2022-08-17,2023-06-09,NaT,NaT,NaT,NaT,0.000000,1.0,0.000000
2,202401,DOC-00001222,4,2,0,4,2,2024-01-23,2024-01-25,NaT,NaT,2024-01-23,2024-01-25,0.500000,1.0,0.500000
3,202401,DOC-00001255,2,1,0,2,1,2022-05-09,2022-05-20,NaT,NaT,2022-05-20,2022-05-20,0.500000,1.0,0.500000
4,202401,DOC-00001452,2,0,0,2,0,2023-10-10,2023-10-26,NaT,NaT,NaT,NaT,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7391094,202502,TLF-999989491,17,2,0,17,2,2022-05-16,2023-05-16,NaT,NaT,2023-05-11,2023-05-16,0.117647,1.0,0.117647
7391095,202502,TLF-999989522,3,2,0,3,2,2022-06-02,2023-03-23,NaT,NaT,2022-06-02,2022-11-11,0.666667,1.0,0.666667
7391096,202502,TLF-999989548,7,2,0,7,2,2024-12-14,2025-03-12,NaT,NaT,2024-12-14,2024-12-17,0.285714,1.0,0.285714
7391097,202502,TLF-999989830,5,0,0,5,0,2023-09-18,2023-09-22,NaT,NaT,NaT,NaT,0.000000,1.0,0.000000


In [ ]:
df_leads_tipif = pd.merge( df_leads_periodo ,df_tipif_periodo , how = 'left' , on = [ 'periodo' , 'persona_id' ] )

df_leads_tipif_doc = pd.merge( df_leads_tipif , df_pers_alumno[ ~df_pers_alumno['documento'].isna() ].groupby( [ 'persona_id' ] , as_index = False )['documento'].last() , how = 'left' , on = [ 'persona_id' ] )
df_leads_tipif_codalumno = pd.merge( df_leads_tipif , df_pers_alumno[ ~df_pers_alumno['cod_alumno'].isna() ].groupby( [ 'persona_id' ] , as_index = False )['cod_alumno'].last() , how = 'left' , on = [ 'persona_id' ] )

df_leads_tipif_doc = df_leads_tipif_doc.groupby( [ 'periodo' , 'documento' ] , as_index = False ).agg(
        cant_lead_ids = ( 'cant_lead_ids' , 'sum' ) ,
        cant_tipif = ( 'cant_tipif' , 'sum' ) ,
        cant_tipif_contact_efect = ( 'cant_tipif_contact_efect' , 'sum' ) ,
        cant_tipif_no_comerc = ( 'cant_tipif_no_comerc' , 'sum' ) ,
        cant_tipif_comerc = ( 'cant_tipif_comerc' , 'sum' ) ,
        cant_tipif_comerc_contact_efect = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
        fecha_prim_tipif = ( 'fecha_prim_tipif' , 'min' ) ,
        fecha_ult_tipif = ( 'fecha_ult_tipif' , 'max' ) ,
        fecha_prim_tipif_no_comerc = ( 'fecha_prim_tipif_no_comerc' , 'min' ) ,
        fecha_ult_tipif_no_comerc = ( 'fecha_ult_tipif_no_comerc' , 'max' ) ,
        fecha_prim_tipif_comerc_contact_efect = ( 'fecha_prim_tipif_comerc_contact_efect' , 'min' ) ,
        fecha_ult_tipif_comerc_contact_efect = ( 'fecha_ult_tipif_comerc_contact_efect' , 'max' ) ,
)
df_leads_tipif_doc['ratio_tipif_contact_efect'] = df_leads_tipif_doc['cant_tipif_contact_efect'] /df_leads_tipif_doc['cant_tipif']
df_leads_tipif_doc['ratio_tipif_comerc'] = df_leads_tipif_doc['cant_tipif_comerc'] /df_leads_tipif_doc['cant_tipif']
df_leads_tipif_doc['ratio_tipif_comerc_contact_efect'] = df_leads_tipif_doc['cant_tipif_comerc_contact_efect'] /df_leads_tipif_doc['cant_tipif_comerc']

df_leads_tipif_codalumno = df_leads_tipif_codalumno.groupby( [ 'periodo' , 'cod_alumno' ] , as_index = False ).agg(
        cant_lead_ids = ( 'cant_lead_ids' , 'sum' ) ,
        cant_tipif = ( 'cant_tipif' , 'sum' ) ,
        cant_tipif_contact_efect = ( 'cant_tipif_contact_efect' , 'sum' ) ,
        cant_tipif_no_comerc = ( 'cant_tipif_no_comerc' , 'sum' ) ,
        cant_tipif_comerc = ( 'cant_tipif_comerc' , 'sum' ) ,
        cant_tipif_comerc_contact_efect = ( 'cant_tipif_comerc_contact_efect' , 'sum' ) ,
        fecha_prim_tipif = ( 'fecha_prim_tipif' , 'min' ) ,
        fecha_ult_tipif = ( 'fecha_ult_tipif' , 'max' ) ,
        fecha_prim_tipif_no_comerc = ( 'fecha_prim_tipif_no_comerc' , 'min' ) ,
        fecha_ult_tipif_no_comerc = ( 'fecha_ult_tipif_no_comerc' , 'max' ) ,
        fecha_prim_tipif_comerc_contact_efect = ( 'fecha_prim_tipif_comerc_contact_efect' , 'min' ) ,
        fecha_ult_tipif_comerc_contact_efect = ( 'fecha_ult_tipif_comerc_contact_efect' , 'max' ) ,
)
df_leads_tipif_codalumno['ratio_tipif_contact_efect'] = df_leads_tipif_codalumno['cant_tipif_contact_efect'] /df_leads_tipif_codalumno['cant_tipif']
df_leads_tipif_codalumno['ratio_tipif_comerc'] = df_leads_tipif_codalumno['cant_tipif_comerc'] /df_leads_tipif_codalumno['cant_tipif']
df_leads_tipif_codalumno['ratio_tipif_comerc_contact_efect'] = df_leads_tipif_codalumno['cant_tipif_comerc_contact_efect'] /df_leads_tipif_codalumno['cant_tipif_comerc']

df_leads_tipif = pd.concat( [ df_leads_tipif_doc , df_leads_tipif_codalumno ] , axis = 0 ).reset_index( drop = True )

df_leads_tipif

,periodo,documento,cant_lead_ids,cant_tipif,cant_tipif_contact_efect,cant_tipif_no_comerc,cant_tipif_comerc,cant_tipif_comerc_contact_efect,fecha_prim_tipif,fecha_ult_tipif,fecha_prim_tipif_no_comerc,fecha_ult_tipif_no_comerc,fecha_prim_tipif_comerc_contact_efect,fecha_ult_tipif_comerc_contact_efect,ratio_tipif_contact_efect,ratio_tipif_comerc,ratio_tipif_comerc_contact_efect,cod_alumno
0,202401,00001000,3,4.0,3.0,0.0,4.0,3.0,2023-08-08,2024-02-15,NaT,NaT,2023-08-08,2023-08-08,0.750000,1.0,0.750000,NaN
1,202401,00001008,4,17.0,9.0,0.0,17.0,9.0,2023-06-09,2023-10-24,NaT,NaT,2023-06-09,2023-09-29,0.529412,1.0,0.529412,NaN
2,202401,00001085,3,5.0,0.0,0.0,5.0,0.0,2022-08-17,2023-06-09,NaT,NaT,NaT,NaT,0.000000,1.0,0.000000,NaN
3,202401,00001110,4,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN
4,202401,00001177,1,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7948372,202502,NaN,2,6.0,2.0,0.0,6.0,2.0,2022-09-17,2023-02-14,NaT,NaT,2022-09-17,2023-02-10,0.333333,1.0,0.333333,N00544077
7948373,202502,NaN,1,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,N00544112
7948374,202502,NaN,1,1.0,0.0,0.0,1.0,0.0,2024-09-27,2024-09-27,NaT,NaT,NaT,NaT,0.000000,1.0,0.000000,N00544128
7948375,202502,NaN,2,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,N00544193


# DATASET FINAL

In [50]:
df_final = df_leads_tipif.copy()

cols_final = [
    'periodo',
    'documento',
    'cod_alumno',
    'cant_lead_ids',
    'cant_tipif',
    'cant_tipif_contact_efect',
    'cant_tipif_no_comerc',
    'cant_tipif_comerc',
    'cant_tipif_comerc_contact_efect',
    'fecha_prim_tipif',
    'fecha_ult_tipif',
    'fecha_prim_tipif_no_comerc',
    'fecha_ult_tipif_no_comerc',
    'fecha_prim_tipif_comerc_contact_efect',
    'fecha_ult_tipif_comerc_contact_efect',
    'ratio_tipif_contact_efect',
    'ratio_tipif_comerc',
    'ratio_tipif_comerc_contact_efect',
]
df_final = df_final[ cols_final ].copy()

df_final

,periodo,documento,cod_alumno,cant_lead_ids,cant_tipif,cant_tipif_contact_efect,cant_tipif_no_comerc,cant_tipif_comerc,cant_tipif_comerc_contact_efect,fecha_prim_tipif,fecha_ult_tipif,fecha_prim_tipif_no_comerc,fecha_ult_tipif_no_comerc,fecha_prim_tipif_comerc_contact_efect,fecha_ult_tipif_comerc_contact_efect,ratio_tipif_contact_efect,ratio_tipif_comerc,ratio_tipif_comerc_contact_efect
0,202401,00001000,NaN,3,4.0,3.0,0.0,4.0,3.0,2023-08-08,2024-02-15,NaT,NaT,2023-08-08,2023-08-08,0.750000,1.0,0.750000
1,202401,00001008,NaN,4,17.0,9.0,0.0,17.0,9.0,2023-06-09,2023-10-24,NaT,NaT,2023-06-09,2023-09-29,0.529412,1.0,0.529412
2,202401,00001085,NaN,3,5.0,0.0,0.0,5.0,0.0,2022-08-17,2023-06-09,NaT,NaT,NaT,NaT,0.000000,1.0,0.000000
3,202401,00001110,NaN,4,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
4,202401,00001177,NaN,1,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7948372,202502,NaN,N00544077,2,6.0,2.0,0.0,6.0,2.0,2022-09-17,2023-02-14,NaT,NaT,2022-09-17,2023-02-10,0.333333,1.0,0.333333
7948373,202502,NaN,N00544112,1,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
7948374,202502,NaN,N00544128,1,1.0,0.0,0.0,1.0,0.0,2024-09-27,2024-09-27,NaT,NaT,NaT,NaT,0.000000,1.0,0.000000
7948375,202502,NaN,N00544193,2,0.0,0.0,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN


In [53]:
pq.write_table( pa.Table.from_pandas( df_final ) , ruta_output_pq )